In [1]:
# Import Libraries
import pandas as pd
import lightgbm as lgbm
import catboost as catb
from joblib import load

import sklearn.metrics as metric

C:\Users\ejose\AppData\Roaming\Python\Python311\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
# Load Models
default_tb_lgbm = load('./GBDT_Training/Outputs/Results/Demo/LGBM/Train (Default)/DEMO_LGBM_TB.model') # <== Point these to the respective .model files
default_ib_lgbm = load('./GBDT_Training/Outputs/Results/Demo/LGBM/Train (Default)/DEMO_LGBM_IB.model')
tuned_tb_lgbm = load('./GBDT_Training/Outputs/Results/Demo/LGBM/Train (Tuned)/TUNED_DEMO_LGBM_TB.model')
tuned_ib_lgbm = load('./GBDT_Training/Outputs/Results/Demo/LGBM/Train (Tuned)/TUNED_DEMO_LGBM_IB.model')

default_tb_catb = catb.CatBoostClassifier()
default_ib_catb = catb.CatBoostClassifier()
tuned_tb_catb = catb.CatBoostClassifier()
tuned_ib_catb = catb.CatBoostClassifier()
default_tb_catb = default_tb_catb.load_model("./GBDT_Training/Outputs/Results/Demo/CATB/Train (Default)//DEMO_CATB_TB.model", format='json') # <== Point these to the respective .model files
default_ib_catb = default_ib_catb.load_model("./GBDT_Training/Outputs/Results/Demo/CATB/Train (Default)/DEMO_CATB_IB.model", format='json')
tuned_tb_catb = tuned_tb_catb.load_model("./GBDT_Training/Outputs/Results/Demo/CATB/Train (Tuned)/TUNED_DEMO_CATB_TB.model", format='json')
tuned_ib_catb = tuned_ib_catb.load_model("./GBDT_Training/Outputs/Results/Demo/CATB/Train (Tuned)/TUNED_DEMO_CATB_IB.model", format='json')

c:\Users\ejose\anaconda3\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator LabelEncoder from version 1.2.1 when using version 1.4.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [3]:
# Load Test/Holdout Datasets

DF_LGBM_TB = pd.read_csv('./Dataset/TB/LGBM_TB_Test.csv', low_memory=False) #<== Point these to the proper Test/Holdout datasets.
DF_LGBM_IB = pd.read_csv('./Dataset/IB/LGBM_IB_Test.csv', low_memory=False)

DF_CATB_TB = pd.read_csv('./Dataset/TB/CATB_TB_Test.csv', low_memory=False) #<== Point these to the proper Test/Holdout datasets.
DF_CATB_IB = pd.read_csv('./Dataset/IB/CATB_IB_Test.csv', low_memory=False)

DF_CATB_TB = DF_CATB_TB.fillna("NaN")
DF_CATB_IB = DF_CATB_IB.fillna("NaN")

In [4]:
print("Default TB LGBM")
print(metric.classification_report(DF_LGBM_TB['malware'], default_tb_lgbm.predict(DF_LGBM_TB.iloc[:,1:101]),digits=4))

print("Tuned TB LGBM")
print(metric.classification_report(DF_LGBM_TB['malware'], tuned_tb_lgbm.predict(DF_LGBM_TB.iloc[:,1:101]),digits=4))

Default TB LGBM
[LightGBM] [Warning] Unknown parameter: categorical_data
              precision    recall  f1-score   support

           0     0.9194    0.5481    0.6867       104
           1     0.9891    0.9988    0.9940      4284

    accuracy                         0.9881      4388
   macro avg     0.9542    0.7735    0.8404      4388
weighted avg     0.9875    0.9881    0.9867      4388

Tuned TB LGBM
              precision    recall  f1-score   support

           0     0.9016    0.5288    0.6667       104
           1     0.9887    0.9986    0.9936      4284

    accuracy                         0.9875      4388
   macro avg     0.9452    0.7637    0.8301      4388
weighted avg     0.9866    0.9875    0.9859      4388



In [5]:
print("Default IB LGBM")
print(metric.classification_report(DF_LGBM_IB['malware'], default_ib_lgbm.predict(DF_LGBM_IB.iloc[:,1:101]),digits=4))
print("Tuned IB LGBM")
print(metric.classification_report(DF_LGBM_IB['malware'], tuned_ib_lgbm.predict(DF_LGBM_IB.iloc[:,1:101]),digits=4))

Default IB LGBM
[LightGBM] [Warning] Unknown parameter: categorical_data
              precision    recall  f1-score   support

           0     0.8904    0.6250    0.7345       104
           1     0.9910    0.9981    0.9945      4284

    accuracy                         0.9893      4388
   macro avg     0.9407    0.8116    0.8645      4388
weighted avg     0.9886    0.9893    0.9884      4388

Tuned IB LGBM
              precision    recall  f1-score   support

           0     0.8182    0.5192    0.6353       104
           1     0.9884    0.9972    0.9928      4284

    accuracy                         0.9859      4388
   macro avg     0.9033    0.7582    0.8140      4388
weighted avg     0.9844    0.9859    0.9843      4388



In [6]:
print("Default TB CATB")
print(metric.classification_report(DF_CATB_TB['malware'], default_tb_catb.predict(DF_CATB_TB.iloc[:,1:101]),digits=4))
print("Tuned TB CATB")
print(metric.classification_report(DF_CATB_TB['malware'], tuned_tb_catb.predict(DF_CATB_TB.iloc[:,1:101]),digits=4))

Default TB CATB
              precision    recall  f1-score   support

           0     0.9524    0.5769    0.7186       104
           1     0.9898    0.9993    0.9945      4284

    accuracy                         0.9893      4388
   macro avg     0.9711    0.7881    0.8566      4388
weighted avg     0.9889    0.9893    0.9880      4388

Tuned TB CATB
              precision    recall  f1-score   support

           0     0.9394    0.5962    0.7294       104
           1     0.9903    0.9991    0.9947      4284

    accuracy                         0.9895      4388
   macro avg     0.9648    0.7976    0.8620      4388
weighted avg     0.9891    0.9895    0.9884      4388



In [7]:
print("Default IB CATB")
print(metric.classification_report(DF_CATB_IB['malware'], default_ib_catb.predict(DF_CATB_IB.iloc[:,1:101]),digits=4))
print("Tuned IB CATB")
print(metric.classification_report(DF_CATB_IB['malware'], tuned_ib_catb.predict(DF_CATB_IB.iloc[:,1:101]),digits=4))

Default IB CATB
              precision    recall  f1-score   support

           0     0.9189    0.6538    0.7640       104
           1     0.9917    0.9986    0.9951      4284

    accuracy                         0.9904      4388
   macro avg     0.9553    0.8262    0.8796      4388
weighted avg     0.9899    0.9904    0.9896      4388

Tuned IB CATB
              precision    recall  f1-score   support

           0     0.9242    0.5865    0.7176       104
           1     0.9901    0.9988    0.9944      4284

    accuracy                         0.9891      4388
   macro avg     0.9571    0.7927    0.8560      4388
weighted avg     0.9885    0.9891    0.9879      4388

